# Sentiment Analysis 

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

In [4]:
df = pd.read_excel('Data.xlsx')
df.clean_text=df.clean_text.astype(str)

In [5]:
df.head()

,clean_text,category
0,when modi promised â€œminimum government maxim...,-1
1,talk all the nonsense and continue all the dra...,0
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1


In [6]:
df.columns

Index(['clean_text', 'category'], dtype='object')

In [7]:
df = df[df['category'] != 0.0 ]
print(df.shape)
df.head(5)

(978, 2)


,clean_text,category
0,when modi promised â€œminimum government maxim...,-1
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1
8,with upcoming election india saga going import...,1


In [8]:
df['category'] = df['category'].replace(to_replace={-1:'Negative',1:'Positive'}) 

In [9]:
df.dtypes

clean_text    object
category      object
dtype: object

In [10]:
df["category"].value_counts()

Positive    653
Negative    325
Name: category, dtype: int64

In [11]:
sentiment_label = df.category.factorize()
sentiment_label

(array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 

In [12]:
tweet = df.clean_text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [13]:
print(tokenizer.word_index)

{'modi': 1, 'the': 2, 'and': 3, 'for': 4, 'you': 5, 'will': 6, 'india': 7, 'are': 8, 'that': 9, 'this': 10, 'not': 11, 'with': 12, 'but': 13, 'has': 14, 'his': 15, 'bjp': 16, 'have': 17, 'from': 18, 'all': 19, 'like': 20, 'who': 21, 'govt': 22, 'was': 23, 'people': 24, 'vote': 25, 'more': 26, 'our': 27, 'congress': 28, 'your': 29, 'now': 30, 'they': 31, 'why': 32, 'one': 33, 'can': 34, 'dont': 35, 'only': 36, 'time': 37, 'their': 38, 'how': 39, 'even': 40, 'should': 41, 'after': 42, 'years': 43, 'its': 44, 'what': 45, 'any': 46, 'country': 47, 'government': 48, 'there': 49, 'narendra': 50, 'just': 51, 'about': 52, 'him': 53, 'rahul': 54, 'against': 55, 'again': 56, 'when': 57, 'good': 58, 'new': 59, 'power': 60, 'chowkidar': 61, 'than': 62, 'also': 63, 'which': 64, 'modis': 65, 'make': 66, 'get': 67, 'via': 68, 'indian': 69, 'election': 70, 'sir': 71, 'party': 72, 'better': 73, 'want': 74, 'elections': 75, 'best': 76, 'out': 77, 'nation': 78, '2019': 79, 'would': 80, 'great': 81, 'unde

In [14]:
print(tweet[0])
print(encoded_docs[0])

when modi promised â€œminimum government maximum governanceâ€ expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples
[57, 1, 523, 2191, 48, 2192, 2193, 631, 53, 1301, 2, 1302, 210, 2194, 2, 139, 32, 168, 83, 43, 67, 937, 139, 41, 3, 11, 938, 3, 41, 1303, 1304, 3, 939]


In [15]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0   57
    1  523 2191   48 2192 2193  631   53 1301    2 1302  210 2194    2
  139   32  168   83   43   67  937  139   41    3   11  938    3   41
 1303 

In [16]:
embedding_vector_length = 32
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])  
print(model.summary()) 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           175968    
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 192,619
Trainable params: 192,619
Non-trainable params: 0
__________________________________________________

In [17]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=10, batch_size=32)

Epoch 1/10
25/25 [==============================] - 16s 388ms/step - loss: 0.6666 - accuracy: 0.6624 - val_loss: 0.6257 - val_accuracy: 0.6684
Epoch 2/10
25/25 [==============================] - 9s 375ms/step - loss: 0.6368 - accuracy: 0.6675 - val_loss: 0.6228 - val_accuracy: 0.6684
Epoch 3/10
25/25 [==============================] - 7s 283ms/step - loss: 0.6116 - accuracy: 0.6675 - val_loss: 0.6130 - val_accuracy: 0.6684
Epoch 4/10
25/25 [==============================] - 8s 336ms/step - loss: 0.5624 - accuracy: 0.6701 - val_loss: 0.5867 - val_accuracy: 0.6684
Epoch 5/10
25/25 [==============================] - 9s 362ms/step - loss: 0.3878 - accuracy: 0.8491 - val_loss: 0.6876 - val_accuracy: 0.7143
Epoch 6/10
25/25 [==============================] - 5s 186ms/step - loss: 0.3399 - accuracy: 0.8875 - val_loss: 0.5857 - val_accuracy: 0.7092
Epoch 7/10
25/25 [==============================] - 10891s 454s/step - loss: 0.2094 - accuracy: 0.9361 - val_loss: 0.5301 - val_accuracy: 0.7500
Ep

In [ ]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", sentiment_label[1][prediction])

In [ ]:
test_sentence1 = "Disappointed with the BJP governments by therir specific policy or action"
predict_sentiment(test_sentence1)